In [31]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
appName("Sneha Spark Session").\
config("spark.shuffle.useOldFetchProtocol", 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [32]:
spark

In [33]:
order_schema = 'order_id long, order_date string, customer_id long, order_status string'

In [34]:
order_df = spark.read.\
format("csv").\
schema(order_schema).\
load("/public/trendytech/orders/orders_1gb.csv")

In [35]:
order_df.show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|       98

In [36]:
spark.sparkContext.defaultParallelism

2

In [37]:
order_df.rdd.getNumPartitions()

9

In [38]:
order_df.createOrReplaceTempView("orders")

In [39]:
spark.sql("select count(distinct(order_status)) from orders")

count(DISTINCT order_status)
9


In [40]:
spark.sql("select count(*) from orders where order_status = 'CLOSED'").show()

+--------+
|count(1)|
+--------+
| 2833500|
+--------+



In [41]:
spark.sql("select order_status,count(*) from orders group by order_status").show()

+---------------+--------+
|   order_status|count(1)|
+---------------+--------+
|PENDING_PAYMENT| 5636250|
|       COMPLETE| 8587125|
|        ON_HOLD| 1424250|
| PAYMENT_REVIEW|  273375|
|     PROCESSING| 3103125|
|         CLOSED| 2833500|
|SUSPECTED_FRAUD|  584250|
|        PENDING| 2853750|
|       CANCELED|  535500|
+---------------+--------+

